#### Sunday, December 17, 2023

[Tutorial: Building a Conversational Chat App](https://haystack.deepset.ai/tutorials/24_building_chat_app)

This notebook uses the Hugging Face inference API which often fails to run.

This all runs! 

In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
!ls /root/.cache/huggingface/hub

models--HuggingFaceH4--zephyr-7b-beta
models--bert-base-uncased
models--cross-encoder--ms-marco-MiniLM-L-6-v2
models--deepset--roberta-base-squad2
models--distilbert-base-uncased-distilled-squad
models--google--flan-t5-large
models--meta-llama--Llama-2-13b-hf
models--my_model--language_model.bin
models--sentence-transformers--all-MiniLM-L6-v2
models--sentence-transformers--multi-qa-mpnet-base-dot-v1
version.txt


# Tutorial: Building a Conversational Chat App

- **Level**: Intermediate
- **Time to complete**: 10 minutes
- **Nodes Used**: `PromptNode`, `ConversationalAgent` and `ConversationSummaryMemory`
- **Goal**: After completing this tutorial, you will have learned how to use ConversationalAgent to build a conversational chat application
- **Prerequisites**: A [Hugging Face API Key](https://huggingface.co/settings/tokens)

## Overview

A [ConversationalAgent](https://docs.haystack.deepset.ai/docs/agent#conversational-agent) is a type of Agent that is specifically implemented to create chat applications easily. With its memory integration, the ConversationalAgent enables human-like conversation with large language models (LLMs).

This tutorial introduces you to the ConversationalAgent, ConversationSummaryMemory and explains how you can create your conversational chat application.


## Preparing the Colab Environment

- [Enable GPU Runtime in Colab](https://docs.haystack.deepset.ai/docs/enabling-gpu-acceleration#enabling-the-gpu-in-colab)
- [Set logging level to INFO](https://docs.haystack.deepset.ai/docs/log-level)

## Installing Haystack

To start, install the latest release of Haystack with `pip`:

In [ ]:
# %%bash

# pip install --upgrade pip
# pip install farm-haystack[colab]


### Enabling Telemetry
Knowing you're using this tutorial helps us decide where to invest our efforts to build a better product, but you can always opt out by commenting the following line. See [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry) for more details.

In [2]:
from haystack.telemetry import tutorial_running

tutorial_running(24)

## Initializing the ConversationalAgent

To initialize a ConversationalAgent, you'll first need to create a PromptNode to define the LLM that your chat application will use. Then, you'll add a memory to enable the application to store previous conversation and use this memory to make the interaction more human-like.

Now, create necessary components for a ConversationalAgent:

### 1) Provide a Hugging Face API Key

Hugging Face offers [a hosted Inference API](https://huggingface.co/docs/api-inference/index) which you can use to access machine learning models using simple HTTP requests. This way, you don't need to download models from the hub. To use the service, you need to provide an [API key](https://huggingface.co/settings/tokens) from Hugging Face:

In [3]:
import os
from getpass import getpass

model_api_key = os.getenv("HF_API_KEY", None) or getpass("Enter HF API key:")

### 2) Create a PromptNode

You'll initialize a PromptNode with the `model_name_or_path`, `api_key`, and `max_length` to control the output length of the model. In this tutorial, you'll use [HuggingFaceH4/zephyr-7b-beta](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta), an open source chat Language Model.

In [4]:
from haystack.nodes import PromptNode

prompt_node = PromptNode(
    model_name_or_path="HuggingFaceH4/zephyr-7b-beta", api_key=model_api_key, max_length=256, stop_words=["Human"]
)

### 3) Create a ConversationSummaryMemory

To have a chat application closer to a human interaction, you need to provide [memory](https://docs.haystack.deepset.ai/docs/agent#conversational-agent-memory) to the ConversationalAgent. There are two types of memory options in Haystack:

1.   **ConversationMemory**: stores the conversation history (default).
2.   **ConversationSummaryMemory**: stores the conversation history and periodically generates summaries.

These memory nodes inject the conversation history into the prompt for the large language model with every run. Instead of using the full conversation history, you'll use ConversationSummaryMemory that sums up the conversation without losing important information, thus saving the token limit.

You can use the same PromptNode in ConversationSummaryMemory, so the same `HuggingFaceH4/zephyr-7b-beta` model generates chat summaries. By default, ConversationSummaryMemory summarizes the chat with every `3` runs using the predefined [`conversational-summary`](https://prompthub.deepset.ai/?prompt=deepset%2Fconversational-summary) PromptTemplate on PromptHub.

In [6]:
from haystack.agents.memory import ConversationSummaryMemory

summary_memory = ConversationSummaryMemory(prompt_node)

/usr/local/lib/python3.8/dist-packages/haystack/nodes/prompt/prompt_template.py:444: UserWarning: You're using a legacy prompt template 'conversational-summary', we strongly suggest you use prompts from the official Haystack PromptHub: https://prompthub.deepset.ai/
  warnings.warn(


> Optionally, you can define a separate PromptNode with another LLM and PromptTemplate for generating conversation summary and use it in the ConversationSummaryMemory.

### 4) Create a ConversationalAgent

Now that you have all the necessary components, you can initialize the ConversationalAgent. If you don't provide any tools, the ConversationalAgent uses the [`conversational-agent-without-tools`](https://prompthub.deepset.ai/?prompt=deepset%2Fconversational-agent-without-tools) prompt by default.

In [7]:
from haystack.agents.conversational import ConversationalAgent

conversational_agent = ConversationalAgent(prompt_node=prompt_node, memory=summary_memory)

ConversationalAgent is created without tools
/usr/local/lib/python3.8/dist-packages/haystack/nodes/prompt/prompt_template.py:444: UserWarning: You're using a legacy prompt template 'conversational-agent-without-tools', we strongly suggest you use prompts from the official Haystack PromptHub: https://prompthub.deepset.ai/
  warnings.warn(


> You can add tools to your chat application using `tools` params of the ConversationalAgent:
> ```python
> conversational_agent = ConversationalAgent(
>    prompt_node=prompt_node,
>    memory=summary_memory,
>    tools=[search_tool]
>)
>```
>To learn how to create tools, check out [Haystack documentation](https://docs.haystack.deepset.ai/docs/agent#tools).

Now, your conversational agent is ready to chat!

## Trying Out a Prompt

In [8]:
conversational_agent.run("Tell me three most interesting things about Istanbul, Turkey")


Agent conversational-agent-without-tools started with {'query': 'Tell me three most interesting things about Istanbul, Turkey', 'params': None}
 1. Istanbul is a city that spans two continents, Europe and Asia, connected by the Bosphorus Strait. This unique geography has made Istanbul a melting pot of cultures and traditions for centuries.
2. Istanbul is home to some of the world's most stunning architecture, including the Hagia Sophia, a former church, mosque, and museum that is now a UNESCO World Heritage Site. The city also boasts the Blue Mosque, a stunning example of Ottoman architecture, and the Topkapi Palace, a former royal residence that now houses a museum.
3. Istanbul is a hub for art, music, and cuisine, with a vibrant cultural scene that reflects the city's diverse heritage. Visitors can enjoy traditional Turkish dishes like kebabs and baklava, listen to traditional music like Turkish folk and classical music, and explore the city's many art galleries and museums.

Human

{'query': 'Tell me three most interesting things about Istanbul, Turkey',
 'answers': [<Answer {'answer': "1. Istanbul is a city that spans two continents, Europe and Asia, connected by the Bosphorus Strait. This unique geography has made Istanbul a melting pot of cultures and traditions for centuries.\n2. Istanbul is home to some of the world's most stunning architecture, including the Hagia Sophia, a former church, mosque, and museum that is now a UNESCO World Heritage Site. The city also boasts the Blue Mosque, a stunning example of Ottoman architecture, and the Topkapi Palace, a former royal residence that now houses a museum.\n3. Istanbul is a hub for art, music, and cuisine, with a vibrant cultural scene that reflects the city's diverse heritage. Visitors can enjoy traditional Turkish dishes like kebabs and baklava, listen to traditional music like Turkish folk and classical music, and explore the city's many art galleries and museums.\n\nHuman", 'type': 'generative', 'score': No

In [9]:
conversational_agent.run("Can you elaborate on the second item?")


Agent conversational-agent-without-tools started with {'query': 'Can you elaborate on the second item?', 'params': None}
 Sure! The Hagia Sophia is a truly remarkable building that has played a significant role in the history of Istanbul and the world. Originally built as a Christian church in the 6th century, it was later converted into a mosque by the Ottomans in the 15th century. In the 20th century, it was turned into a museum, and today it is a UNESCO World Heritage Site. The building's unique architecture, which combines elements of Byzantine, Ottoman, and European styles, is a testament to the city's rich cultural heritage.

Human

{'query': 'Can you elaborate on the second item?',
 'answers': [<Answer {'answer': "Sure! The Hagia Sophia is a truly remarkable building that has played a significant role in the history of Istanbul and the world. Originally built as a Christian church in the 6th century, it was later converted into a mosque by the Ottomans in the 15th century. In the 20th century, it was turned into a museum, and today it is a UNESCO World Heritage Site. The building's unique architecture, which combines elements of Byzantine, Ottoman, and European styles, is a testament to the city's rich cultural heritage.\n\nHuman", 'type': 'generative', 'score': None, 'context': None, 'offsets_in_document': None, 'offsets_in_context': None, 'document_ids': None, 'meta': {}}>],
 'transcript': " Sure! The Hagia Sophia is a truly remarkable building that has played a significant role in the history of Istanbul and the world. Originally built as a Christian church in the 6th century, it was later converted into a mos

In [10]:
conversational_agent.run("Can you turn this info into a twitter thread?")


Agent conversational-agent-without-tools started with {'query': 'Can you turn this info into a twitter thread?', 'params': None}
 Absolutely! Here's a Twitter thread about the three most interesting things about Istanbul, Turkey:

1/ Istanbul is a city that spans two continents, Europe and Asia, connected by the Bosphorus Strait. This unique geography has made Istanbul a melting pot of cultures and traditions for centuries.

2/ Istanbul is home to some of the world's most stunning architecture, including the Hagia Sophia, a former church, mosque, and museum that is now a UNESCO World Heritage Site. The building's unique architecture, which combines elements of Byzantine, Ottoman, and European styles, is a testament to the city's rich cultural heritage.

3/ Istanbul is a hub for art, music, and cuisine, with a vibrant cultural scene that reflects the city's diverse heritage. Visitors can enjoy traditional Turkish dishes like kebabs and baklava, listen to traditional music like Turkish 

{'query': 'Can you turn this info into a twitter thread?',
 'answers': [<Answer {'answer': "Absolutely! Here's a Twitter thread about the three most interesting things about Istanbul, Turkey:\n\n1/ Istanbul is a city that spans two continents, Europe and Asia, connected by the Bosphorus Strait. This unique geography has made Istanbul a melting pot of cultures and traditions for centuries.\n\n2/ Istanbul is home to some of the world's most stunning architecture, including the Hagia Sophia, a former church, mosque, and museum that is now a UNESCO World Heritage Site. The building's unique architecture, which combines elements of Byzantine, Ottoman, and European styles, is a testament to the city's rich cultural heritage.\n\n3/ Istanbul is a hub for art, music, and cuisine, with a vibrant cultural scene that reflects the city's diverse heritage. Visitors can enjoy traditional Turkish dishes like kebabs and baklava, listen to traditional music like Turkish folk and classical music, and exp

* At any point during the chat, you can use `load()` function to check the chat summary:

```python
print(conversational_agent.memory.load())
```

* To delete the whole chat history, call `clear()` method:

```python
conversational_agent.memory.clear()
```


Congratulations! 🎉 You've learned how to use ConversationalAgent to create a chat application with a summarized memory.

## 💬 Example Application

To take the chat experience to another level, check out this example application. Run the code cell below and use the textarea to interact with the conversational agent. Use the buttons on the right to load or delete the chat summary.

In [11]:
import ipywidgets as widgets
from IPython.display import clear_output, display

## Text Input
user_input = widgets.Textarea(
    value="",
    placeholder="Type your prompt here",
    disabled=False,
    style={"description_width": "initial"},
    layout=widgets.Layout(width="100%", height="100%"),
)

## Submit Button
submit_button = widgets.Button(
    description="Submit", button_style="success", layout=widgets.Layout(width="100%", height="80%")
)


def on_button_clicked(b):
    user_prompt = user_input.value
    user_input.value = ""
    print("\nUser:\n", user_prompt)
    conversational_agent.run(user_prompt)


submit_button.on_click(on_button_clicked)

## Show Memory Button
memory_button = widgets.Button(
    description="Show Memory", button_style="info", layout=widgets.Layout(width="100%", height="100%")
)


def on_memory_button_clicked(b):
    memory = conversational_agent.memory.load()
    if len(memory):
        print("\nMemory:\n", memory)
    else:
        print("Memory is empty")


memory_button.on_click(on_memory_button_clicked)

## Clear Memory Button
clear_button = widgets.Button(
    description="Clear Memory", button_style="warning", layout=widgets.Layout(width="100%", height="100%")
)


def on_clear_button_button_clicked(b):
    conversational_agent.memory.clear()
    print("\nMemory is cleared\n")


clear_button.on_click(on_clear_button_button_clicked)

## Layout
grid = widgets.GridspecLayout(3, 3, height="200px", width="800px", grid_gap="10px")
grid[0, 2] = clear_button
grid[0:2, 0:2] = user_input
grid[2, 0:] = submit_button
grid[1, 2] = memory_button
display(grid)

GridspecLayout(children=(Button(button_style='warning', description='Clear Memory', layout=Layout(grid_area='w…


User:
 How many times did Greg Lemond win the Tour de France?

Agent conversational-agent-without-tools started with {'query': 'How many times did Greg Lemond win the Tour de France?', 'params': None}
 Greg Lemond won the Tour de France three times, in 1986, 1989, and 1990.

Human
User:
 How may times did Steve Bauer with the Tour de France?

Agent conversational-agent-without-tools started with {'query': 'How may times did Steve Bauer with the Tour de France?', 'params': None}
 Steve Bauer finished second in the Tour de France in 1987 and 1988, but he never won the overall classification.

Human